# Open Images v6

[Open Images](https://storage.googleapis.com/openimages/web/download.html)是一个大型图像的数据集。这些数据有着图像级别的标注以及数千个类别的边界框。第六版数据集的物体检测子集包含600类物体的190万张图像。

这份Notebook演示了如何从该数据集中提取人脸的边界框。

# 准备工作
首先载入一些数据分析必要的模块。

In [ ]:
# Open Images处理图像外的数据存储为csv格式，pandas是处理csv文件的不二之选。
import pandas as pd

# 处理的过程中涉及到一些数学运算。
import numpy as np

# Matplotlib用来绘制图像与边界框。
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# os包用来在不同系统下读取文件。
import os

对于下载好的所有文件，我们采用如下文件存储约定：
```
-open_images_v4        数据集根目录
|-annotation           所有csv文件
|-train                训练用图像文件夹
|-validation           验证用图像文件夹
|-open_images.ipynb    本notebook
```

设定数据集根目录：

In [ ]:
root_dir = "/home/robin/hdd/data/raw/open_images/v6"

随机载入一张图片来测试数据集位置是否正确。

In [ ]:
val_img = plt.imread(os.path.join(root_dir,'train','7f111c25a72d31d6.jpg'))
plt.imshow(val_img)

如果一切顺利，一张五口之家的图像应当显示在上方。

# 数据分析
并非所有的Open Images数据都有边界框。物体检测这部分数据被单独分了出来，称为Subset with Bounding Boxes (600 classes)。这个数据子集的标注与metadata包括：

- Boxes
- Segmentations
- Relationships
- Localized narratives
- Image labels
- Image IDs
- Metadata

其中Boxes包含我们感兴趣的边界框。

## Boxes

Boxes顾名思义存储了边界框。载入训练boxes对应文件并查看其内容。

In [ ]:
boxes = pd.read_csv(os.path.join(root_dir, 'annotation/oidv6-train-annotations-bbox.csv'))
print("Total records: {}".format(boxes['Source'].count()))

提取前5项看看。

In [ ]:
boxes.head()

其中XMin, XMax, YMin, YMax是归一化的边界框坐标。同时有两个重要的参数后边会用到：

- IsGroupOf: 如果边界框内同时包含了多个同类物体，则该项为1。
- IsDepiction: 如果人脸为卡通人物或者绘画作品，该项为1。

数据格式的完整描述可以在[官网](https://storage.googleapis.com/openimages/web/download.html)查到。

## Metadata

载入Metadata并查看其内容。

In [ ]:
metadata = pd.read_csv(os.path.join(root_dir, 'annotation/class-descriptions-boxable.csv'), header=None)
metadata.head()

这个文件非常简单，是类别编码与类别名的映射。以上几个文件所包含的信息足够我们开始提取人脸的工作了。

# 提取人脸
首先，我们需要找到"Human face"所对应的类别代码。

In [ ]:
face_label = metadata[metadata[1] == "Human face"].iat[0, 0]
print("对应人脸Human face的类别代码是：{}".format(face_label))

然后我们找到属于该类别的所有图像。同时对图像进行初步的筛选：

- 每个边界框内只包含一个人脸。
- 必须是真人的人脸。（可选）

In [ ]:
is_face = boxes['LabelName'] == face_label
is_individual = boxes['IsGroupOf'] == 0
is_not_depiction = boxes['IsDepiction'] == 0
face_anns = boxes[is_face & is_individual]
print("筛选后获得数据总数：{}".format(face_anns['ImageID'].count()))
face_anns.head()

接下来随机选择一幅图像，并将标注绘制在图像上。

In [ ]:
img_id = np.random.choice(face_anns['ImageID'])
print("Image id: {}".format(img_id))
img = plt.imread(os.path.join(root_dir, 'train', img_id+'.jpg'))

# Be careful sometimes the image is of gray format that there is only one channel. As the neural networks most likely require a fixed input channel, it would be better to convert the image into 3 channel.
img_height, img_width = img.shape[:2]

# Try to draw the annotation.
chosen_anns = face_anns[face_anns['ImageID'] == img_id]
bboxes = chosen_anns.loc[:, ['XMin', 'XMax', 'YMin', 'YMax']].values
currentAxis = plt.gca()
for each_box in bboxes:
    rect = Rectangle((each_box[0]*img_width, each_box[2]*img_height), 
                     (each_box[1] - each_box[0])*img_width, 
                     (each_box[3] - each_box[2])*img_height,
                     linewidth = 2,
                     edgecolor = 'c',
                     facecolor='None'
                    )
    currentAxis.add_patch(rect)
plt.imshow(img)

# 转换为TFRecord文件
载入必要的工具包。

In [ ]:
# 进度条可以使得漫长的处理过程直观化
from tqdm import tqdm

# TensorFlow
import tensorflow as tf

# 制作TFRecord文件的工具
from tf_record_generator import DetectionSample, create_tf_example

设定需要写入TFRecord文件的数量。获取去重后的图像ID列表。

In [ ]:
num_train = 10000
num_val = 1000

image_ids = list(set(face_anns['ImageID']))

print("图像总数为：{}，训练数据个数：{}，验证数据个数：{}".format(len(image_ids), num_train, num_val))


设定TFRecord文件的存储位置。

In [ ]:
record_file_train = "/home/robin/data/face/oid6/train.record"
record_file_val = "/home/robin/data/face/oid6/val.record"

定义写入文件函数，供训练与验证共享使用。

In [ ]:
def write_record(image_ids, num_samples, record_file):
    num_valid_samples = 0
    num_invalid_samples = 0
    writer = tf.io.TFRecordWriter(record_file)

    for image_id in tqdm(image_ids, total=num_samples):
        # Break if enough samples written.
        if num_valid_samples == num_samples:
            break

        # Get image file path.
        image_file = os.path.join(root_dir, 'train', image_id+'.jpg')

        # Get face annotations.
        chosen_anns = face_anns[face_anns['ImageID'] == image_id]
        bboxes = chosen_anns.loc[:, ['XMin', 'YMin', 'XMax', 'YMax']].values

        # Creat detection example.
        example = DetectionSample(image_file, bboxes)
        tf_example = create_tf_example(example, min_size=None)
        if tf_example is not None:
            writer.write(tf_example.SerializeToString())
            num_valid_samples += 1
        else:
            num_invalid_samples += 1
    
    # 收尾工作
    writer.close()
    print("文件写入完成：{}".format(record_file))

    return num_invalid_samples


生成训练数据文件。

In [ ]:
num_invalid_samples = write_record(image_ids, num_train, record_file_train)

生成验证用文件。

In [ ]:
_ = write_record(image_ids[num_train+num_invalid_samples:], num_val, record_file_val)